In [1]:
from utils.utils import read_yaml
from utils.model_factory import load_model
import torch

cfg = read_yaml('projects/configs/cfg_LymphPath.yaml')
model = load_model(cfg)
model.load_state_dict(torch.load('weights/LymphPath.pt'), strict=True)

Using KAN
success to init LymphPath


<All keys matched successfully>

In [2]:
import pandas as pd
import random
from datasets.ThreeStreamBagDataset import ThreeChannelBagDataset

random.seed(1)
your_csv = pd.read_csv('csv/17-BRCA.csv')

wsi_id = random.choice(list(your_csv['slide_id']))
wsi_data = your_csv[your_csv['slide_id'] == wsi_id]

data_dir1 = '/feature/gigapath/'
data_dir2 = '/feature/UNI/'
data_dir3 = '/feature/virchow2/'

dataset = ThreeChannelBagDataset(
    df=wsi_data,
    data_dir1=data_dir1,
    data_dir2=data_dir2,
    data_dir3=data_dir3,
    label_field='label'
)

sample = dataset[0]
feature1 = sample['features1']  
feature2 = sample['features2']
feature3 = sample['features3']
y = sample['label']


In [3]:
model.eval()
model.to('cuda')
feature1 = feature1.to('cuda')
feature2 = feature2.to('cuda')
feature3 = feature3.to('cuda')
y =y .to('cuda')
res = model(feature1, feature2, feature3, label=y, instance_eval=True)

logits1 = res['logits1']
logits2 = res['logits2']
logits3 = res['logits3']
bag_logits_merge = res['merge_logits']

y_prob1 = torch.softmax(logits1, dim=-1)
y_prob2 = torch.softmax(logits2, dim=-1)
y_prob3 = torch.softmax(logits3, dim=-1)
y_prob_merge = torch.softmax(bag_logits_merge, dim=-1)
y_prob = 0.4 * (y_prob1 + y_prob2 + y_prob3) / 3 + 0.6 * y_prob_merge
print('slide id:', str(wsi_id), 'if LNM:', int(y_prob[:,1] > 0.5))

slide id: patient_013_node_3 if LNM: 1
